In [10]:
import pandas as pd
# Read dataset
extreme_weather = pd.read_csv('crime_extreme_weather_CLEANED.csv')

extreme_weather.head()

,Unnamed: 0,Complaint_No,Property_Crimes,Location_Type,Zipcode,Property_No,Property_type,Property_Desc,2010 Pop,Median Household Income,DATE,EVENT_TYPE,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,time_diff,weather_id
0,2732,20120701220002,Shoplifting,Indoors,28216,1.0,Alcoholic Beverage,Consumable Goods,47227,53462.0,2012-07-01,Hail,35.133333,-80.740767,35.133333,-80.740767,0 days 00:16:00,30.0
1,2733,20120701220002,Shoplifting,Indoors,28216,1.0,Alcoholic Beverage,Consumable Goods,47227,53462.0,2012-07-01,Lightning,35.340000,-80.810000,35.340000,-80.810000,0 days 00:15:00,59.0
2,2734,20120701220002,Shoplifting,Indoors,28216,1.0,Alcoholic Beverage,Consumable Goods,47227,53462.0,2012-07-01,Thunderstorm Wind,35.320000,-80.800000,35.180000,-80.655000,0 days 00:19:00,82.0
3,2738,20120701224501,Arson,Indoors,28205,1.0,Dwelling: Multi-Family,Structures,43931,61525.0,2012-07-01,Hail,35.133333,-80.740767,35.133333,-80.740767,0 days 00:16:00,30.0
4,2739,20120701224501,Arson,Indoors,28205,1.0,Dwelling: Multi-Family,Structures,43931,61525.0,2012-07-01,Lightning,35.340000,-80.810000,35.340000,-80.810000,0 days 00:15:00,59.0


In [11]:
extreme_weather = extreme_weather[['Property_Crimes', 'Property_type', 'EVENT_TYPE', 'Location_Type', 'Property_Desc', 'Median Household Income', '2010 Pop']]
print(extreme_weather.head())
extreme_weather.dropna(inplace=True)
extreme_weather.reset_index(drop=True, inplace=True)
print(extreme_weather.shape)

  Property_Crimes           Property_type         EVENT_TYPE Location_Type  \
0     Shoplifting      Alcoholic Beverage               Hail       Indoors   
1     Shoplifting      Alcoholic Beverage          Lightning       Indoors   
2     Shoplifting      Alcoholic Beverage  Thunderstorm Wind       Indoors   
3           Arson  Dwelling: Multi-Family               Hail       Indoors   
4           Arson  Dwelling: Multi-Family          Lightning       Indoors   

      Property_Desc  Median Household Income  2010 Pop  
0  Consumable Goods                  53462.0     47227  
1  Consumable Goods                  53462.0     47227  
2  Consumable Goods                  53462.0     47227  
3        Structures                  61525.0     43931  
4        Structures                  61525.0     43931  
(3213, 7)


In [12]:
extreme_weather = pd.get_dummies(extreme_weather, columns=['Property_type', 'EVENT_TYPE', 'Location_Type', 'Property_Desc'])


In [26]:
# create training and testing sets - 80/20
from sklearn.model_selection import train_test_split
crime_train, crime_test = train_test_split(extreme_weather, test_size =0.20)
print(crime_test.shape)
print(crime_train.shape)

(643, 81)
(2570, 81)


In [27]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

x = crime_train.drop(columns='Property_Crimes')
y = crime_train['Property_Crimes']

In [28]:
cart01 = DecisionTreeClassifier(max_leaf_nodes=10).fit(x,y) 
rf01 = RandomForestClassifier(n_estimators = 10,criterion="gini").fit(x,y)

In [29]:
prediction_cart01 = cart01.predict(x)
prediction_rf01 = rf01.predict(x)

In [30]:
print(crime_train['Property_Crimes'].count())
crime_train['Property_Crimes'].value_counts(normalize=True)

2570


Burglary/B&E                                 0.589883
Shoplifting                                  0.205447
Motor Vehicle Theft                          0.143191
Theft of Motor Vehicle Parts from Vehicle    0.054475
Arson                                        0.004280
Embezzlement                                 0.002724
Name: Property_Crimes, dtype: float64

In [31]:
# create a function that we can use to check how our predictions compare to the actual values
def eval_prediction(pred, actual):
    index = 0
    correct = 0
    for outcome in actual:
        if pred[index] == outcome:
            correct += 1

        index+=1
    return correct

In [32]:
#Using the evaluation function to see how many we got correct for each model 
print("CART:", eval_prediction(prediction_cart01, y))
print("Random Forest:", eval_prediction(prediction_rf01, y))

CART: 2118
Random Forest: 2448


In [33]:
print("CART:", '{0:.2f}'.format((eval_prediction(prediction_cart01, y)/len(x))*100),"%")
print("Random Forest:", '{0:.2f}'.format((eval_prediction(prediction_rf01, y)/len(x))*100),"%")

CART: 82.41 %
Random Forest: 95.25 %


In [34]:
# Now we would want to see how well our models perfom on the testing set
x_test = crime_test.drop(columns='Property_Crimes')
y_test = crime_test['Property_Crimes']


print(rf01.score(x_test,y_test))
print(cart01.score(x_test,y_test))

0.8771384136858476
0.8087091757387247


In [36]:
#After further evaluation, Our random forest model is performing significantly higher than our cart model. 
#The Random Forest model is performing significantly higher than the Cart model, but that is to be expected as the Random Forest model is much more complex.
#The Cart model on the other hand, helps us better understand whether or not our variables are improving the accuracy of our predictions because of its simplicity.

